In [1]:
import pandas as pd
import re
import math
import time

In [2]:
# All global Variables
ssd_structure = {
        "channel": 2,
        "chip": 1,
        "die": 2,
        "plane": 4,
        "block_container": 60,
        "block": 5,
    }
total_host_write = 0
total_ssd_write = 0
total_gc_write = 0
max_write_count_global = 0
ssd_block_trace_dict = {}
ssd_block_trace_list = []
lba_block_trace_dict = {}

In [3]:
def gc():
    write_trace_list = []
    for block_id in ssd_block_trace_list:
        if ssd_block_trace_dict[block_id]['bs'] == 2:
            if len(ssd_block_trace_dict[block_id]['lba']) == 0:
                ssd_block_trace_dict[block_id]['gcs'] += 1
                ssd_block_trace_dict[block_id]['bs'] = 0
                ssd_block_trace_dict[block_id]['wpc'] = 0
                ssd_block_trace_dict[block_id]['dpc'] = 0
                ssd_block_trace_dict[block_id]['ec'] += 1
                ssd_block_trace_dict[block_id]['ds'] = 0
                ssd_block_trace_dict[block_id]['aw'] = 0
                ssd_block_trace_dict[block_id]['lba'] = {}
            else:
                for lba in ssd_block_trace_dict[block_id]['lba']:
                    write_trace_list.append({
                        'lba': lba,
                        'io_s': ssd_block_trace_dict[block_id]['lba'][lba] * 4 * 1000,
                        'gcs': 1
                    })
                ssd_block_trace_dict[block_id]['gcs'] += 1
                ssd_block_trace_dict[block_id]['bs'] = 0
                ssd_block_trace_dict[block_id]['wpc'] = 0
                ssd_block_trace_dict[block_id]['dpc'] = 0
                ssd_block_trace_dict[block_id]['ec'] += 1
                ssd_block_trace_dict[block_id]['ds'] = 0
                ssd_block_trace_dict[block_id]['aw'] = 0
                ssd_block_trace_dict[block_id]['lba'] = {}
    
    return write_trace_list

In [4]:
def allocation_scheme_algorithm(allocation_scheme, block_tracer):
    global ssd_structure
    if allocation_scheme == 's1':
        channel = math.floor(block_tracer / (ssd_structure['plane'] * ssd_structure['die'] * ssd_structure['chip'])) % ssd_structure['channel']
        chip = block_tracer % ssd_structure['chip']
        die = math.floor(block_tracer / ssd_structure['chip']) % ssd_structure['die']
        plane = math.floor(block_tracer / (ssd_structure['die'] * ssd_structure['chip'])) % ssd_structure['plane']
        block_container = (math.floor(block_tracer / (ssd_structure['plane'] * ssd_structure['die'] * ssd_structure['chip'] * ssd_structure['channel'])) % ssd_structure['block_container'])
        block = math.floor(block_tracer / (ssd_structure['plane'] * ssd_structure['die'] * ssd_structure['chip'] * ssd_structure['channel'] * ssd_structure['block_container'])) % ssd_structure['block']
        block_id = "block" + "_" + str(channel) + "_" + str(chip) + "_" + str(die) + "_" + str(plane) + "_" + str(block_container) + "_" + str(block)
        return block_id
    elif allocation_scheme == 's2':
        channel = block_tracer % ssd_structure['channel']
        chip = math.floor(block_tracer / ssd_structure['channel']) % ssd_structure['chip']
        die = math.floor(block_tracer / (ssd_structure['chip'] * ssd_structure['channel'])) % ssd_structure['die']
        plane = math.floor(block_tracer / (ssd_structure['die'] * ssd_structure['chip'] * ssd_structure['channel'])) % ssd_structure['plane']
        block_container = (math.floor(block_tracer / (ssd_structure['plane'] * ssd_structure['die'] * ssd_structure['chip'] * ssd_structure['channel'])) % ssd_structure['block_container'])
        block = math.floor(block_tracer / (ssd_structure['plane'] * ssd_structure['die'] * ssd_structure['chip'] * ssd_structure['channel'] * ssd_structure['block_container'])) % ssd_structure['block']
        block_id = "block" + "_" + str(channel) + "_" + str(chip) + "_" + str(die) + "_" + str(plane) + "_" + str(block_container) + "_" + str(block)
        return block_id
    else:
        return None

In [5]:
def write_block(allocation_scheme, traces):
    start_time = time.time()
    block_tracer = 0
    trace_list_tracer = 0
    global ssd_block_trace_dict
    global ssd_block_trace_list
    global lba_block_trace_dict
    global total_host_write
    global total_ssd_write
    global total_gc_write
    def delete_from_ssd(lba):
        
        if lba in lba_block_trace_dict:
            for block_id in lba_block_trace_dict[lba]:
                ssd_block_trace_dict[block_id]['dpc'] += ssd_block_trace_dict[block_id]['lba'][lba]
                ssd_block_trace_dict[block_id]['lba'].pop(lba)
                lba_block_trace_dict[lba].remove(block_id)
                ssd_block_trace_dict[block_id]['ds'] += 1
                ssd_block_trace_dict[block_id]['bs'] = 2
                    
    def add_to_ssd(block_id, io_size, devisable_by_4, lba):
        global max_write_count_global
        if block_id in ssd_block_trace_dict:
            ssd_block_trace_dict[block_id]['aw'] += io_size
            ssd_block_trace_dict[block_id]['wpc'] += devisable_by_4
            ssd_block_trace_dict[block_id]['wc'] += 1
            ssd_block_trace_dict[block_id]['lba'][lba] = ssd_block_trace_dict[block_id]['lba'][lba] + 1 if lba in ssd_block_trace_dict[block_id]['lba'] else 1
            if ssd_block_trace_dict[block_id]['wc'] > max_write_count_global:
                max_write_count_global = ssd_block_trace_dict[block_id]['wc']
        else:
            ssd_block_trace_dict[block_id] = {
                'aw': io_size,
                'dpc': 0,
                'wpc': devisable_by_4,
                'wc': 1,
                'bs': 0,
                'ds': 0,
                'lba': {
                    # lba: 0 if lba not in ssd_block_trace_dict else ssd_block_trace_dict[lba]
                    lba: 1
                    },
                'ec': 0,
                'gcs': 0
            }
            # append only block_id to ssd_block_trace_list if block_id is not in ssd_block_trace_list
            if block_id not in ssd_block_trace_list:
                ssd_block_trace_list.append(block_id)
        # print("Block ID: ", ssd_block_trace_dict[block_id])


    def trace_lba(block_id, lba):
        if lba not in lba_block_trace_dict:
            lba_block_trace_dict[lba] = [block_id]
        else:
            if block_id not in lba_block_trace_dict[lba]:
                lba_block_trace_dict[lba].append(block_id)
    while trace_list_tracer < len(traces):
        
        if 'gcs' not in traces[trace_list_tracer]:
            delete_from_ssd(traces[trace_list_tracer]['lba'])
            total_host_write += int(traces[trace_list_tracer]['io_s'])
        else:
            total_gc_write += traces[trace_list_tracer]['io_s']
            
        block_id = allocation_scheme_algorithm(allocation_scheme, block_tracer)
        # print(block_id)
        io_size = int(traces[trace_list_tracer]['io_s'])/1000
        # find how many times io_size is devisable by 4 and what is remainder 
        devisable_by_4 = io_size // 4
        remainder = io_size % 4
        if remainder > 0:
            devisable_by_4 += 1
        
        while devisable_by_4 > 0:
            aw = 4 if io_size >= 4 else io_size

            # if (block_tracer % 10000) == 0:
            #     print("block id: ", block_tracer)
            #     print("io_s",devisable_by_4)
            if block_id in ssd_block_trace_dict:
                # print("io_s1",devisable_by_4)
                # print(ssd_block_trace_dict[block_id])
                if ssd_block_trace_dict[block_id]['bs'] == 1:
                    block_tracer += 1
                else:
                    add_to_ssd(block_id, aw, 1, traces[trace_list_tracer]['lba'])
                    trace_lba(block_id, traces[trace_list_tracer]['lba'])
                    devisable_by_4 -= 1
                    io_size -= 4
                    block_id = allocation_scheme_algorithm(allocation_scheme, block_tracer)
                
                    if ssd_block_trace_dict[block_id]['wpc'] >= 256:
                        ssd_block_trace_dict[block_id]['bs'] = 1
                        block_tracer += 1
                        block_id = allocation_scheme_algorithm(allocation_scheme, block_tracer)
            else:
                # print("io_s1",devisable_by_4)
                
                add_to_ssd(block_id, aw, 1, traces[trace_list_tracer]['lba'])
                trace_lba(block_id, traces[trace_list_tracer]['lba'])
                devisable_by_4 -= 1
                io_size -= 4
                block_id = allocation_scheme_algorithm(allocation_scheme, block_tracer)
            
                if ssd_block_trace_dict[block_id]['wpc'] >= 256:
                    ssd_block_trace_dict[block_id]['bs'] = 1
                    block_tracer += 1
                    block_id = allocation_scheme_algorithm(allocation_scheme, block_tracer)
            total_ssd_write += 4000
            if time.time() - start_time > 20:
                break
        
        if time.time() - start_time > 20:
            print(total_ssd_write)
            break
        
        trace_list_tracer += 1
        #  check the progra
        if total_ssd_write > 4886394000:
            print("Total Host Write: ", total_host_write)
            print("Total SSD Write: ", total_ssd_write)
            print("Total GC Write: ", total_gc_write)
            print("average waf", (total_ssd_write+total_gc_write)/total_host_write)
            time.sleep(4)
            trace_list = gc()
            write_block('s1', trace_list)
            
            
    
    return {
        'ssd_block_trace_dict': ssd_block_trace_dict,
        'ssd_block_trace_list': ssd_block_trace_list,
    }
    


In [6]:
file_path = 'trace_files/test_trace.txt'

with open(file_path, 'r', encoding='utf-8') as input_file:
    write_trace_list = []
    fields_name = ['lba', 'io_s']  
    
    while True:
        return_line = {}
        try:
            line = next(input_file)
            # line = next(input_file)
            if not line:
                break
            line = line.rstrip('\n')
            line = line.strip()
            line_split = line.split(' ')            
            for index, field_value in enumerate(line_split):
                return_line[fields_name[index]] = field_value
            write_trace_list.append(return_line)
        
            
        except StopIteration:
            break

return_value = write_block('s1', write_trace_list)
trace_list = gc()
write_block('s1', trace_list)
# trace_list = gc()
# write_block('s1', trace_list)
print("Total Host Write: ", total_host_write)
print("Total SSD Write: ", total_ssd_write)
print("Total GC Write: ", total_gc_write)
print("average waf", (total_ssd_write+total_gc_write)/total_host_write)

print(ssd_block_trace_dict)




Total Host Write:  9527296
Total SSD Write:  16348000
Total GC Write:  5376000
average waf 2.28018526977644
{'block_0_0_0_0_0_0': {'aw': 1024, 'dpc': 0, 'wpc': 256, 'wc': 512, 'bs': 1, 'ds': 0, 'lba': {'67520': 2, '3408888': 131, '3409904': 3, '3411952': 3, '3412984': 117}, 'ec': 1, 'gcs': 1}, 'block_0_0_1_0_0_0': {'aw': 1024, 'dpc': 0, 'wpc': 256, 'wc': 512, 'bs': 1, 'ds': 0, 'lba': {'3412984': 14, '3414000': 3, '7372672': 17, '5801408': 5, '5801440': 5, '5801472': 5, '5801504': 5, '5801536': 5, '5801568': 5, '5801600': 5, '5801632': 5, '5801664': 5, '5807328': 4, '5801696': 5, '5801728': 5, '5801760': 5, '5801792': 5, '5801824': 5, '5801856': 5, '5801888': 5, '5801920': 5, '5801952': 5, '5801984': 5, '5802016': 5, '5802048': 5, '5802080': 5, '5802112': 5, '5804288': 3, '5802144': 5, '5802176': 5, '5807648': 5, '5802208': 5, '5802240': 5, '5802272': 5, '5807680': 5, '5802304': 5, '5807712': 5, '5807744': 5, '5802336': 5, '5807776': 5, '5802368': 5, '5807808': 5, '5807840': 5, '5802400